# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_temp,Wind_speed
0,0,Sarkand,91,KZ,1586642392,36,45.41,79.92,47.91,4.14
1,1,Jamestown,1,US,1586642384,48,42.10,-79.24,48.00,14.99
2,2,New Norfolk,75,AU,1586642394,57,-42.78,147.06,52.00,13.87
3,3,Port Alfred,77,ZA,1586642200,90,-33.59,26.89,68.00,4.00
4,4,Saint-Pierre,40,RE,1586642297,73,-21.34,55.48,75.20,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = cities.loc[(cities["Max_temp"] > 70) & (cities["Max_temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_temp,Wind_speed
0,41,Pisco,0,PE,1586642444,30,-13.70,-76.22,75.99,5.82
1,123,Ambilobe,0,MG,1586642550,84,-13.20,49.05,73.67,4.72
2,132,Mossamedes,0,AO,1586642561,61,-15.20,12.15,77.59,3.62
3,150,Tura,0,IN,1586642462,32,25.52,90.22,72.18,4.45
4,316,Tsabong,0,BW,1586642792,30,-26.02,22.40,74.77,12.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_temp,Wind_speed,Hotel Name
0,41,Pisco,0,PE,1586642444,30,-13.70,-76.22,75.99,5.82,Pisco
1,123,Ambilobe,0,MG,1586642550,84,-13.20,49.05,73.67,4.72,Ambilobe
2,132,Mossamedes,0,AO,1586642561,61,-15.20,12.15,77.59,3.62,Namibe
3,150,Tura,0,IN,1586642462,32,25.52,90.22,72.18,4.45,Tura
4,316,Tsabong,0,BW,1586642792,30,-26.02,22.40,74.77,12.12,Tshabong


In [7]:
hotel_df=narrowed_city_df


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
locations

,Lat,Lng
0,-13.70,-76.22
1,-13.20,49.05
2,-15.20,12.15
3,25.52,90.22
4,-26.02,22.40
5,-21.14,-55.83
6,10.47,79.13
7,-22.47,-68.93
8,14.73,78.55
9,-23.13,-64.33


In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))